# Explore data from the python client

## Relevant documents

- (Python Client Repo)[https://github.com/Green-Fusion/energy-management-backend/tree/main/python_client]
- (Swagger UI)[https://control.green-fusion.de/services/energy-management-backend/v1/api/doc#/]


## Inits

In [1]:
#imports
from energy_management_client import BackendPythonClient
import pandas as pd
import json
from datetime import datetime
import pprint  # For pretty-printing

In [3]:
# select your environment
environment="prod" # prod or dev

# Selector
if environment=="dev":
    backend_endpoint_path="https://dev.green-fusion.de/services/energy-management-backend/v1/api"
    realm_type="development"
elif environment=="prod":
    backend_endpoint_path="https://control.green-fusion.de/services/energy-management-backend/v1/api"
    realm_type="control"  
else:
    backend_endpoint_path=""
    realm_type=""
    print("select either prod or dev for the environment")

#test
#backend_endpoint_path, realm_type
if True:
    # pythpon client auth & login
    # init the client   
    client = BackendPythonClient(backend_endpoint=backend_endpoint_path,)
    
    #login
    client.login(
        auth_endpoint="https://auth.green-fusion.de",
        realm=realm_type,
        client_id="backend_python_client", # backend_python_client, datascience_development_api
        grant_type="device_code", #"client_credentials",  # or "password", "device_code"
    )

## Functions

### Customers

In [4]:
def customers_to_df(customer_list):
    """
    Turn a list of CustomerDto objects into a pandas DataFrame.
    usage:
        customer_list = client.customers.get_list_of_customers()
        df_customers = customers_to_df(customer_list)
    
    """
    rows = []

    for cust in customer_list:
        rows.append({
            "id":        cust.id,
            "name":      cust.name,
            "gui_name":  getattr(cust, "gui_name", None),
            "external_id": getattr(cust, "external_id", None),
            "f_dashboard": cust.f_dashboard,
            "f_meldungen": cust.f_meldungen,
            "f_meldungen_v2": cust.f_meldungen_v2,
            "f_optimisation": cust.f_optimisation,
            "f_technique": cust.f_technique,
            "notification_email_contacts": getattr(cust, "notification_email_contacts", []),
        })

    return pd.DataFrame(rows)

### buildings

In [5]:
def get_buildings_df(client, customers):
    """
    Loop through all customers, fetch their buildings, and build a DataFrame where each row contains:
      - building_id, customerID (from customer object), address, city, atvise_display_name, base,
      - heatingType, heatingSurface, access_key, nickname, postal_code, gui_name,
      - coordinates, CustomerAlias, gfid,
      - LayoutID (extracted from modular_system)

    Usage:
        customers = client.customers.get_list_of_customers()
        df = get_buildings_df(client, customers)
        print(df.head())
    """
    try:
        data = []
        for c in customers:
            customer_id = c.id
            customer_name = getattr(c, 'name', None)
            client.configure(customer_id=customer_id)
            
            # Get buildings
            c_buildings = client.buildings.get_buildings()
            building_list = getattr(c_buildings, 'buildings', [])
            
            for b in building_list:
                building_id         = getattr(b, 'building_id', None)
                address             = getattr(b, 'address', None)
                city                = getattr(b, 'city', None)
                atvise_display_name = getattr(b, 'atvise_display_name', None)
                base                = getattr(b, 'base', None)
                heatingType         = getattr(b, 'heatingType', None)
                heatingSurface      = getattr(b, 'heatingSurface', None)
                access_key          = getattr(b, 'access_key', None)
                nickname            = getattr(b, 'nickname', None)
                postal_code         = getattr(b, 'postal_code', None)
                gui_name            = getattr(b, 'gui_name', None)
                coordinates         = getattr(b, 'coordinates', None)
                CustomerAlias       = getattr(b, 'CustomerAlias', None)
                gfid                = getattr(b, 'gfid', None)
                
                # Parse modular_system
                modular_system_str = getattr(b, 'modular_system', None)
                LayoutID = None
                if modular_system_str:
                    try:
                        modular_system = json.loads(modular_system_str)
                        LayoutID = modular_system.get('layoutID', None)
                    except json.JSONDecodeError:
                        LayoutID = None

                data.append({
                    "building_id": building_id,
                    "customerID": customer_id,  # <-- corrected here
                    "customer_name": customer_name,
                    "address": address,
                    "city": city,
                    "atvise_display_name": atvise_display_name,
                    "base": base,
                    "heatingType": heatingType,
                    "heatingSurface": heatingSurface,
                    "access_key": access_key,
                    "nickname": nickname,
                    "postal_code": postal_code,
                    "gui_name": gui_name,
                    "coordinates": coordinates,
                    "CustomerAlias": CustomerAlias,
                    "gfid": gfid,
                    "LayoutID": LayoutID
                })

        return pd.DataFrame(data)

    except Exception as e:
        print("API Call Failed:", e)
        return pd.DataFrame()

### Sensors (all)

In [6]:
def get_all_sensors_df(client, building_ids):
    """
    very heavy
    For a list of building IDs, fetch sensor data using client.buildings.get_list_of_sensors_by_building_id
    and return a DataFrame where each row is a sensor.
    
    Parameters:
        client: The API client object
        building_ids: List[int] - list of building IDs to fetch sensors for
        
    Returns:
        pd.DataFrame with one row per sensor, including building_id

    Usage:
        df_sensors = get_all_sensors_df(client, list_of_building_id)
    """
    all_sensors = []

    for b_id in building_ids:
        try:
            sensors = client.buildings.get_list_of_sensors_by_building_id(b_id)
            for s in sensors:
                all_sensors.append({
                    "building_id": b_id,
                    "sensor_id": s.id,
                    "gf_standard_id": s.gf_standard_id,
                    "sub_system_id": s.sub_system_id,
                    "hydraulic_location_index": s.hydraulic_location_index,
                    "factor": s.factor,
                    "current_value": s.current_value,
                    "node_id": s.node_id,
                    "acronym": s.acronym,
                    "unit": s.unit,
                    "long_name": s.long_name,
                    "short_name": s.short_name,
                    "source": s.source,
                })
        except Exception as e:
            print(f"Error fetching sensors for building_id={b_id}: {e}")

    return pd.DataFrame(all_sensors)

### devices

In [7]:
def get_devices(client):
    """
    usage:
    list_device=get_devices(client)
    list(list_device)
    """
    devices_list=client.devices.get_devices()
    devices_list=list(devices_list)[0][1]
    device_dicts = [device.__dict__ for device in devices_list]
    df_device = pd.DataFrame(device_dicts)
    return df_device

### counters

In [21]:
def get_counters(client,ID):
    """
    usage:
    counters_list_for_building=get_counters(client,122)
    """
    counters_list=client.buildings.get_counters(ID)
    return counters_list

def extract_counters_with_errors(client, building_ids):
    all_entries = []

    for building_id in building_ids:
        try:
            counters = get_counters(client, building_id)
        except Exception as e:
            print(f"Failed to get counters for building {building_id}: {e}")
            continue

        for counter in counters:
            if counter.error:  # non-empty string = error exists
                all_entries.append({
                    "id": counter.id,
                    "sensor_id": counter.sensor_id,
                    "reference_date": counter.reference_date,
                    "unit": counter.unit,
                    "error": counter.error,
                    "short_name": counter.short_name,
                    "long_name": counter.long_name,
                    "hydraulic_location_index": counter.hydraulic_location_index
                })

    df = pd.DataFrame(all_entries)
    return df

## Get data

### get_customer_df

In [9]:
customer_list = client.customers.get_list_of_customers()
df_customers = customers_to_df(customer_list)

In [10]:
#df_customers.head()

In [11]:
#df_customers.count()

In [12]:
list_of_customer_id=df_customers['id']

### get_buildings_df

In [13]:
# Get customers list (list of dicts)
customers = client.customers.get_list_of_customers()
#build df
df_buildings = get_buildings_df(client, customers)
#test
#df_buildings.head()

In [14]:
list_of_building_id=df_buildings['building_id']

### get devices

In [19]:
df_devices=get_devices(client)
df_devices = df_devices.rename(columns={"type": "device_type"})

### Merge Buildings <- devices

In [20]:
# Merge the two DataFrames on building_id
df_merged = df_buildings.merge(
    df_devices[['building_id', 'device_type', 'device_id']],
    on='building_id',
    how='left'  # ensures all rows from df_buildings are kept
)

### get all sensors (heavy)

In [15]:
df_sensors = get_all_sensors_df(client, list_of_building_id)

In [16]:
len(df_sensors)

33842

In [17]:
df_sensors.tail()

,building_id,sensor_id,gf_standard_id,sub_system_id,hydraulic_location_index,factor,current_value,node_id,acronym,unit,long_name,short_name,source
33837,448,35164,77.0,219798.0,1.0,0.1,284.000000,3,t_buffer(n)_tank_1,°C,Sensortemperatur 1 Puffer (n),T 1 BT (n),GREENBOX_MQTT
33838,448,35184,403.0,219803.0,1.0,0.1,268.000000,23,t_hp(n)_primflow,°C,Vorlauftemperatur primär Wärmepumpe (n),T VLprim WP (n),GREENBOX_MQTT
33839,755,18516,434.0,NaN,NaN,1.0,15.540000,AT,t_g_out_api,°C,Außentemperatur Global,T A GL,OPEN_WEATHER_API
33840,755,19649,460.0,NaN,NaN,1.0,NaN,YC,e_g_bldg_y,kWh,Energie Gebäude Global Jährlich,E BLDG GL,MANUALLY_ENTERED
33841,755,9498,NaN,NaN,1.0,1.0,679.563049,ZW_ZZ1_rEnergy,None,MWh,None,None,GETEC_MQTT


In [18]:
df_sensors['unit'].value_counts(dropna=False) 

unit
°C       15985
None     12715
kWh       2970
m³         801
kW         641
m³/h       565
%           66
h           19
MWh         16
-           15
W           10
bool        10
 m³/h       10
bar          5
m3/h         4
m°C          3
mbar         3
m3           1
V            1
l/h          1
#            1
Name: count, dtype: int64

### Get counter errors

In [19]:
#get_counters(client,122)

In [22]:
df_errors = extract_counters_with_errors(client, list_of_building_id)

### merge counters <- Sensors

In [98]:
# Merge the error dataframe with the sensor dataframe to get the source
df_merged_counters = df_errors.merge(
    df_sensors[['sensor_id', 'source','building_id','gf_standard_id','acronym','factor']], 
    on='sensor_id', 
    how='left'
)
#df_merged_counters=df_merged_counters.drop(['short_name','long_name'], axis=1)

In [99]:
#reorganize
df_merged_counters['reference_date'] = pd.to_datetime(df_merged_counters['reference_date'])
df_merged_counters=df_merged_counters[[ 'building_id','sensor_id','acronym', 'error',
                                        'reference_date', 'source', 'unit', 'factor',
                                        'hydraulic_location_index',  'gf_standard_id',
                                        'id','short_name','long_name']]

In [101]:
df_merged_counters.tail()

,building_id,sensor_id,acronym,error,reference_date,source,unit,factor,hydraulic_location_index,gf_standard_id,id,short_name,long_name
5344,436,7789,e_hc(n)_hm,Data is empty.,2025-03-31 23:59:59+00:00,GETEC_MQTT,kWh,1.0,2.0,5.0,155850,E WMZ HK (n),Wärme Wärmemengenzähler Heizkreis (n)
5345,436,7789,e_hc(n)_hm,Data is empty.,2023-12-31 23:59:59+00:00,GETEC_MQTT,kWh,1.0,2.0,5.0,155835,E WMZ HK (n),Wärme Wärmemengenzähler Heizkreis (n)
5346,436,7789,e_hc(n)_hm,Data is empty.,2024-03-31 23:59:59+00:00,GETEC_MQTT,kWh,1.0,2.0,5.0,155838,E WMZ HK (n),Wärme Wärmemengenzähler Heizkreis (n)
5347,436,7789,e_hc(n)_hm,Data is empty.,2024-02-29 23:59:59+00:00,GETEC_MQTT,kWh,1.0,2.0,5.0,155837,E WMZ HK (n),Wärme Wärmemengenzähler Heizkreis (n)
5348,436,7789,e_hc(n)_hm,Data is empty.,2024-01-31 23:59:59+00:00,GETEC_MQTT,kWh,1.0,2.0,5.0,155836,E WMZ HK (n),Wärme Wärmemengenzähler Heizkreis (n)


## Work with errors

In [102]:
df_latest_errors = df_merged_counters.loc[df_merged_counters.groupby('sensor_id')['reference_date'].idxmax()].reset_index(drop=True)
df_earliest_errors = df_merged_counters.loc[df_merged_counters.groupby('sensor_id')['reference_date'].idxmin()].reset_index(drop=True)

In [103]:
df_latest_errors.tail()

,building_id,sensor_id,acronym,error,reference_date,source,unit,factor,hydraulic_location_index,gf_standard_id,id,short_name,long_name
444,1850,34151,eel_hp(n)_em_supply_act,No complete data.,2025-05-31 23:59:59+00:00,GREENBOX_MQTT,kWh,0.01,2.0,400.0,163315,Eel supply act SZ WP (n),verbrauchte el. Wirkenergie Stromzähler Wärmep...
445,1698,34446,e_lh(n)_hmsubs,No values greater than 0.,2025-05-31 23:59:59+00:00,GREENBOX_MQTT,kWh,1.00,1.0,465.0,163321,E WMZ SUBS NW (n),Wärme Wärmemengenzähler Unterstation Nahwärmen...
446,1850,34451,eel_hp(n)_em_supply_act,No complete data.,2025-05-31 23:59:59+00:00,GREENBOX_MQTT,kWh,0.01,1.0,400.0,163322,Eel supply act SZ WP (n),verbrauchte el. Wirkenergie Stromzähler Wärmep...
447,2001,34835,eel_hp(n)_em_supply_act,No complete data.,2025-05-31 23:59:59+00:00,GREENBOX_MQTT,kWh,1.00,1.0,400.0,163330,Eel supply act SZ WP (n),verbrauchte el. Wirkenergie Stromzähler Wärmep...
448,2002,34992,eel_hp(n)_em_supply_act,No complete data.,2025-05-31 23:59:59+00:00,GREENBOX_MQTT,kWh,1.00,1.0,400.0,163333,Eel supply act SZ WP (n),verbrauchte el. Wirkenergie Stromzähler Wärmep...


In [104]:
df_earliest_errors.tail()

,building_id,sensor_id,acronym,error,reference_date,source,unit,factor,hydraulic_location_index,gf_standard_id,id,short_name,long_name
444,1850,34151,eel_hp(n)_em_supply_act,No complete data.,2025-05-31 23:59:59+00:00,GREENBOX_MQTT,kWh,0.01,2.0,400.0,163315,Eel supply act SZ WP (n),verbrauchte el. Wirkenergie Stromzähler Wärmep...
445,1698,34446,e_lh(n)_hmsubs,No values greater than 0.,2025-05-31 23:59:59+00:00,GREENBOX_MQTT,kWh,1.00,1.0,465.0,163321,E WMZ SUBS NW (n),Wärme Wärmemengenzähler Unterstation Nahwärmen...
446,1850,34451,eel_hp(n)_em_supply_act,No complete data.,2025-05-31 23:59:59+00:00,GREENBOX_MQTT,kWh,0.01,1.0,400.0,163322,Eel supply act SZ WP (n),verbrauchte el. Wirkenergie Stromzähler Wärmep...
447,2001,34835,eel_hp(n)_em_supply_act,No complete data.,2025-05-31 23:59:59+00:00,GREENBOX_MQTT,kWh,1.00,1.0,400.0,163330,Eel supply act SZ WP (n),verbrauchte el. Wirkenergie Stromzähler Wärmep...
448,2002,34992,eel_hp(n)_em_supply_act,No complete data.,2025-05-31 23:59:59+00:00,GREENBOX_MQTT,kWh,1.00,1.0,400.0,163333,Eel supply act SZ WP (n),verbrauchte el. Wirkenergie Stromzähler Wärmep...


## Exports

In [105]:
#export
if True:
    today_str = datetime.today().strftime('%Y-%m-%d')
    filename = f"df_all_errors_{today_str}.csv"
    filename2 = f"df_earliest_errors_{today_str}.csv"
    filename3 = f"df_latest_errors_{today_str}.csv"
    df_merged_counters.to_csv(filename, index=False)
    df_latest_errors.to_csv(filename2, index=False)
    df_earliest_errors.to_csv(filename3, index=False)
    

## Explore

### Explore get counters

In [78]:
#get_counters(client,122)

### Explore Building status

In [79]:
building_ids = [str(bid) for bid in list_of_building_id[890:910]]
for build in building_ids:
    print(client.buildings.get_status([build]))

[BuildingStatusDto(building_id=2251, sensor_count=10, is_online=True, verbindung=True, yearly_consumption_is_missing=True)]
[BuildingStatusDto(building_id=2252, sensor_count=46, is_online=True, verbindung=True, yearly_consumption_is_missing=True)]
[BuildingStatusDto(building_id=351, sensor_count=9, is_online=True, verbindung=True, yearly_consumption_is_missing=True)]
[BuildingStatusDto(building_id=352, sensor_count=9, is_online=True, verbindung=True, yearly_consumption_is_missing=True)]
[BuildingStatusDto(building_id=576, sensor_count=22, is_online=True, verbindung=True, yearly_consumption_is_missing=False)]
[BuildingStatusDto(building_id=577, sensor_count=26, is_online=True, verbindung=True, yearly_consumption_is_missing=True)]
[BuildingStatusDto(building_id=392, sensor_count=53, is_online=True, verbindung=True, yearly_consumption_is_missing=True)]
[BuildingStatusDto(building_id=1398, sensor_count=38, is_online=True, verbindung=True, yearly_consumption_is_missing=False)]
[BuildingStat

In [92]:
client.buildings.get_status(['711'])[0].is_online

True